Theon-demandelectric scooter is a service in which electric scooters are made available to individuals
 at a cost. It allows people to pick up an electric scooter from one point and return it at another
 point. The user enters payment information, and the computer unlocks a scooter. The user returns
 the scooter, and it gets locked. A locked electric scooter can only be used using a proper control
 mechanism controlled by a mobile application. For many systems, smartphone mapping apps show
 nearby available scooters. Imagine you have been hired as a software developer for such a project by
 an on-demand electric scooter service company.
 Task: The functionality of the on-demand electric scooter-based platform is more or less similar
 to the functionality offered by any other bicycle-sharing platform. Therefore, you can take inspi
ration from other bicycle-sharing software and make suitable assumptions about the functionality
 requirements for the electric scooter application. Make suitable assumptions in case of any missing
 information in the case study. Now, build your application which considers below events:
 1. Develop an on-demand electric scooter-based platform using object-oriented features of Python
 such as assertion, recursion and polymorphism. Use one sorting algorithm for slot management.
 Also, implement exception handling and inheritance.
 2. Automate scooter renting and return processes. Also, focus on the “anywhere” release feature.
 3. Individuals registered with the program identify themselves with their membership card ID at
 any place to check out a scooter for a short period, usually a few hours.
 4. Create multiple scenarios of subscribers renting electric scooters while others release electric
 scooters to stations. Consider a total of 10000 subscribers, 100 stations and 5000 bicycles.
 Remember not all electric scooters are in working status all the time

General assumptions:

Euclidian distance is fine (not haversian)
Rides are paid by the kilometer (5DKK)
Length of rides are assumed to be the manhattan distance between the unlock and the lock 

In [11]:
import numpy as np

price_per_m_DKK = 0.005

The code cell below contains the functionality of the scooter class. They focus on the "anywhere release" functionality, but must be handed in within a reasonable distance of a station. Shortly put, it does the following:

1. Initialization of the scooter class.
2. Scooters can be located anywhere globally.
3. Checks if the scooter is operational and within a 10-meter radius of the user before unlocking.
4. Stores the location before and after the ride to calculate the distance traveled.
5. Ensures the scooter is within a reasonable distance to a station before returning.
6. Assumes the scooter uses 1% of its battery for every 500 meters driven.



In [12]:
class Scooter:
    def __init__(self, scooter_id, position, status='working', station=None):
        self.scooter_id = scooter_id # Unique identifier for each scooter
        self.status = status # Status of the scooter (scooters are initially working)
        self.locked = True # Scooters are initially locked
        self.battery = 1 # Battery level of the scooter, with 1 equalling 100% (scooters are initially fully charged)
        self.station = station # Station where the scooter is currently located. The scooter is only located on a station when it is charging or being repaired
        if isinstance(position, tuple) and len(position) == 2 and all(isinstance(coord, (int, float)) for coord in position): # The position must be a tuple with two numeric values representing global coordinates
            self.position = position
        else:
            raise ValueError("Position must be a tuple with two numeric values representing global coordinates.")
    
    # Unlock the scooter for renting
    def unlock(self, User):
        
        # check if the user is close enough to the scooter (this distance is equivalent to approximately 10 meters)
        distance_to_user = np.linalg.norm(np.array(self.position) - np.array(User.position))
        if distance_to_user > 0.0001: 
            raise ValueError(f"User is too far from scooter {self.scooter_id}. Please get closer to unlock the scooter.") # if the user is too far, raise an error
        elif self.status != 'working':
            raise ValueError(f"Scooter {self.scooter_id} is not working. Please choose another scooter.") # check if the scooter is working
        else:
            self.locked = False # if the scooter is working and nearby, unlock it
        
        # stores the position of the latest unlock, in order to calculate the distance travelled
        self.latest_unlock_position = self.position 
        
        print(f"Scooter {self.scooter_id} is now unlocked and ready for use.")

    def lock(self, station, position): # Lock the scooter when returned
        self.position = position

        # check if the scooter is close enough to the station (this distance is equivalent to approximately 500 meters) before it is locked
        distance_to_station = np.linalg.norm(np.array(self.position) - np.array(station.position))
        if distance_to_station > 0.005: 
            raise ValueError(f"Station is too far from scooter {self.scooter_id}. Please get closer to lock the scooter.")
        else:  
            self.locked = True
        
        # when the ride ends, there is a 1% chance that the scooter will break
        if np.random.rand() < 0.01:
            self.status = 'broken'
            print(f"Scooter {self.scooter_id} is now broken.")
    
        # calculate the distance travelled (useful to calculate the price of the ride and the battery used)
        distance_travelled_m = 0.00001 * (np.abs(self.latest_unlock_position[0] - self.position[0]) + np.abs(self.latest_unlock_position[1] - self.position[1])) #store the manhattan distance between the latest unlock position and the current position in meters
        
        self.battery -= 0.01 * 500*distance_travelled_m # 1% of the battery is used for every 500 meters travelled

        print(f"Scooter {self.scooter_id} is now locked.")

        return distance_travelled_m
    

The cell below defines the station classes. There are two types of stations

1. Recharge stations: can recharge scooters
2. Repair stations: can recharge and repair scooters



In [13]:
class RechargeStation:
    def __init__(self, station_id, position):
        self.station_id = station_id
        self.position = position
        self.scooters = [] # scooters currently located at the station

    # retrieve scooter that is close (within 500m)
    def retrieve_scooter(self, scooter):
        scooter.position = self.position # set the scooter's position to the station's position
        self.scooters.append(scooter) # adds the scooter to the station's list of scooters

    def recharge(self, scooter):
        if scooter in self.scooters:
            scooter.battery = 1 # charge the scooter to 100%

# define a subclass of Station that can also repair scooters
class RepairStation(RechargeStation):
    # repair a broken scooter that is located at the station
    def repair_scooter(self, scooter):
        if scooter in self.scooters and scooter.status != 'working':
            scooter.status = 'working'
        print(f"Scooter {scooter.scooter_id} repaired at station {self.station_id}.")

Then, the user is defined. Shortly put, the class can the following:

1. Defines a method (sorting algorithm) that finds the nearest scooter. Note that it is conscious choice to not use the sorting algorithm for slot management, since we chose to focus on the "release anywhere" function.
2. Defines a mthod for finding the nearest station
3. Account functionality for paying for trips
4. Methods for renting and returning scooters

In [ ]:
class User:
    def __init__(self, membership_id, position):
        self.membership_id = membership_id
        self.position = position
        self.account = 0 # user's account balance for renting bikes. 

    # finds the nearest scooter that is working and charged above 20%
    def find_nearest_scooter(self, scooters):
        scooters = [scooter for scooter in scooters if scooter.status == 'working' and scooter.battery > 0.1] # remove scooters from the list that are not working or low battery
        nearest_scooter = min(scooters, key=lambda scooter: np.linalg.norm(np.array(scooter.position) - np.array(self.position)))  # find the scooter that has the shortest distance to the user
        print(f"Nearest scooter is {nearest_scooter.scooter_id}, which is located at {nearest_scooter.position}.") # helps the user locate the nearest scooter (since they need to be close to unlock it)
        return nearest_scooter
    
    def find_nearest_station(self, scooters):
        stations = [scooter.station for scooter in scooters if scooter.station is not None]
        nearest_station = min(stations, key=lambda station: np.linalg.norm(np.array(station.position) - np.array(self.position)))
        return nearest_station
    
    # method for depositing money into account
    def deposit(self, amount):
        self.account += amount
        print(f"User {self.membership_id} deposited {amount} into their account.")

    # method for renting a scooter
    def rent_scooter(self, scooter):
        scooter.unlock(self)
        print(f"Scooter {scooter.scooter_id} rented by user {self.membership_id}.")

    # method for returning a scooter
    def return_scooter(self, scooter):
        nearest_station = self.find_nearest_station(scooters)
        self.account -= price_per_m_DKK * scooter.lock(nearest_station, self.position) # deduct the price of the ride from the user's account
        if self.account < 0:
            raise ValueError("Insufficient funds. Please deposit more money to your account.")

In [ ]:

# Sample Data for Stations and Scooters
station1 = Station(1, 10)
station2 = Station(2, 10)
scooter1 = Scooter(101, 'working', station1)
scooter2 = Scooter(102, 'working', station2)

# Dictionary to simulate user database
users = {
    "1001": User(1001, "MEM123"),
    "1002": User(1002, "MEM456")
}

stations = {
    "1": station1,
    "2": station2
}

# Input Handling for User Sign-in and Scooter Rent/Return
def sign_in():
    user_id = input("Please enter your User ID: ")
    if user_id in users:
        user = users[user_id]
        print(f"Welcome, User {user.user_id}!")
        return user
    else:
        print("User ID not found.")
        return None

###def rent_scooter_flow(user):
    scooter_id = int(input("Enter the Scooter ID you want to rent: "))
    if scooter_id == scooter1.scooter_id:
        user.rent_scooter(scooter1)
    elif scooter_id == scooter2.scooter_id:
        user.rent_scooter(scooter2)
    else:
        print("Scooter ID not found.")
        
def return_scooter_flow(user):
    scooter_id = int(input("Enter the Scooter ID you are returning: "))
    station_id = input("Enter the station ID where you are returning the scooter: ")

    if scooter_id == scooter1.scooter_id and station_id in stations:
        user.return_scooter(scooter1, stations[station_id])
        stations[station_id].add_scooter(scooter1)
    elif scooter_id == scooter2.scooter_id and station_id in stations:
        user.return_scooter(scooter2, stations[station_id])
        stations[station_id].add_scooter(scooter2)
    else:
        print("Invalid scooter or station ID.")

# Main flow for the simulation
def main():
    user = sign_in()
    if user:
        action = input("Would you like to (1) rent a scooter or (2) return a scooter? Enter 1 or 2: ")
        if action == "1":
            rent_scooter_flow(user)
        elif action == "2":
            return_scooter_flow(user)
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    main()